INSTALL DEPENDENCIES

In [ ]:
!pip install pandas
!pip install scikit-learn
!pip install numpy
!pip install onnxmltools
!pip install onnxruntime
!pip install pyodbc
!pip install setuptools
!pip install skl2onnx
!pip install sqlalchemy

SET VALUES

In [28]:
import pyodbc
import skl2onnx
import sklearn
import sklearn.datasets
import numpy as np
import onnxmltools
import onnxruntime as rt
import pandas as pd
from onnxruntime.capi.onnxruntime_pybind11_state import InvalidArgument


server = 'iotedgevm-f3f741f1-f3f741f1.eastus.cloudapp.azure.com' # SQL Server IP address
username = 'sa' # SQL Server username
password = 'Strong!Passw0rd' # SQL Server password
database = 'IoTEdgeDB'
data_table = 'OpcNodes'
models_table = 'Models'

master_connection_string = "Driver={ODBC Driver 17 for SQL Server};Server=" + server + ";Database=master;UID=" + username + ";PWD=" + password + ";"
db_connection_string = "Driver={ODBC Driver 17 for SQL Server};Server=" + server + ";Database=" + database + ";UID=" + username + ";PWD=" + password + ";"

conn = pyodbc.connect(db_connection_string, autocommit=True)

CREATE X TRAIN DATASET

In [31]:
query = 'SELECT ApplicationUri, DipData, SpikeData, RandomSignedInt32 FROM OpcNodes WHERE DipData IS NOT NULL AND SpikeData IS NOT NULL AND RandomSignedInt32 IS NOT NULL'
sql_query = pd.read_sql_query(query, conn)
x_train = pd.DataFrame(sql_query, columns=['DipData', 'SpikeData', 'RandomSignedInt32'])

print("\n*** Training dataset x\n")
print(x_train.head())


*** Training dataset x

     DipData  SpikeData  RandomSignedInt32
0 -36.812455 -36.812455      -1.574417e+08
1 -36.812455 -36.812455      -1.574417e+08
2 -68.454711 -68.454711      -1.734891e+09
3 -68.454711 -68.454711      -1.734891e+09
4  95.105652  95.105652       4.466349e+08


CREATE Y TRAIN DATASET

In [4]:
target_column = 'MEDV'
y_train = pd.DataFrame()

y_train[target_column] = np.random.randint(100, size=(len(x_train)))

print("\n*** Training dataset y\n")
print(y_train.head())


*** Training dataset y

   MEDV
0    90
1    26
2     2
3     3
4    22


CREATE PIPELINE TO TRAIN THE LINEAR REGRESSION MODEL

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler

continuous_transformer = Pipeline(steps=[('scaler', RobustScaler())])

# All columns are numeric - normalize them
preprocessor = ColumnTransformer(
    transformers = [
        (
            'continuous', 
            continuous_transformer, 
            [i for i in range(len(x_train.columns))]
        )
    ]
)

model = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('regressor', LinearRegression())])

# Train the model
model.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('continuous',
                                                  Pipeline(steps=[('scaler',
                                                                   RobustScaler())]),
                                                  [0, 1, 2])])),
                ('regressor', LinearRegression())])

CHECK MODEL ACCURACY

In [7]:
# Score the model
from sklearn.metrics import r2_score, mean_squared_error
y_pred = model.predict(x_train)
sklearn_r2_score = r2_score(y_train, y_pred)
sklearn_mse = mean_squared_error(y_train, y_pred)
print('*** Scikit-learn r2 score: {}'.format(sklearn_r2_score))
print('*** Scikit-learn MSE: {}'.format(sklearn_mse))

*** Scikit-learn r2 score: 0.0007844648024204481
*** Scikit-learn MSE: 837.4150066212547


DEFINE FUNCTION TO CONVERT MODEL TO ONNX

In [8]:
from skl2onnx.common.data_types import FloatTensorType, Int64TensorType, DoubleTensorType

def convert_dataframe_schema(df, drop=None, batch_axis=False):
    inputs = []
    nrows = None if batch_axis else 1
    for k, v in zip(df.columns, df.dtypes):
        if drop is not None and k in drop:
            continue
        if v == 'int64':
            t = Int64TensorType([nrows, 1])
        elif v == 'float32':
            t = FloatTensorType([nrows, 1])
        elif v == 'float64':
            t = DoubleTensorType([nrows, 1])
        else:
            raise Exception("Bad type")
        inputs.append((k, t))
    return inputs

SAVE MODEL IN ONNX FORMAT

In [9]:
# Convert the scikit model to onnx format
onnx_model = skl2onnx.convert_sklearn(model, 'OPC tags data', convert_dataframe_schema(x_train), final_types=[('variable1',FloatTensorType([1,1]))], target_opset=11)
# Save the onnx model locally
onnx_model_path = 'opcnodes.model.onnx'
onnxmltools.utils.save_model(onnx_model, onnx_model_path)

TEST ONNX MODEL

In [10]:
import onnxruntime as rt
sess = rt.InferenceSession(onnx_model_path)

y_pred = np.full(shape=(len(x_train)), fill_value=np.nan)

for i in range(len(x_train)):
    inputs = {}
    for j in range(len(x_train.columns)):
        inputs[x_train.columns[j]] = np.full(shape=(1,1), fill_value=x_train.iloc[i,j])

    sess_pred = sess.run(None, inputs)
    y_pred[i] = sess_pred[0][0][0]

onnx_r2_score = r2_score(y_train, y_pred)
onnx_mse = mean_squared_error(y_train, y_pred)

print()
print('*** Onnx r2 score: {}'.format(onnx_r2_score))
print('*** Onnx MSE: {}\n'.format(onnx_mse))
print('R2 Scores are equal' if sklearn_r2_score == onnx_r2_score else 'Difference in R2 scores: {}'.format(abs(sklearn_r2_score - onnx_r2_score)))
print('MSE are equal' if sklearn_mse == onnx_mse else 'Difference in MSE scores: {}'.format(abs(sklearn_mse - onnx_mse)))


*** Onnx r2 score: 0.0007844624414192758
*** Onnx MSE: 837.4150085999447

Difference in R2 scores: 2.3610011723462776e-09
Difference in MSE scores: 1.978689965653757e-06



<u>INSERT MODEL IN DATABASE</u>

In [11]:
# Insert the ONNX model into the models table for each value of ApplicationUri
query = f'SELECT DISTINCT ApplicationUri FROM {data_table}'
cursor = conn.cursor()
cursor.execute(query)
application_uris = cursor.fetchall()

for app in application_uris:
    query = f"insert into {models_table} ([applicationUri], [description], [data]) values ('{app[0]}', 'Onnx Model', ?)"
    model_bits = onnx_model.SerializeToString()
    insert_params  = (pyodbc.Binary(model_bits))
    cursor.execute(query, insert_params)
    conn.commit()

RUN PREDICTION

In [37]:

import pandas as pd

conn = pyodbc.connect(db_connection_string, autocommit=True)
cursor = conn.cursor()

# get ONNX model
app_uri = 'urn:OpcPlc:opcserver1'
query = f'SELECT data FROM {models_table} WHERE applicationUri= \'{app_uri}\''
cursor.execute(query)
onnx_model = cursor.fetchall()[0][0]

# get feature data
row_count = 5
query = f'SELECT top {row_count} DipData, SpikeData, RandomSignedInt32 FROM {data_table} WHERE ApplicationUri= \'{app_uri}\' AND DipData IS NOT NULL AND SpikeData IS NOT NULL AND RandomSignedInt32 IS NOT NULL ORDER BY SourceTimestamp DESC'
sql_query = pd.read_sql_query(query, conn)

x_train = pd.DataFrame(sql_query, columns=['DipData', 'SpikeData', 'RandomSignedInt32'])

sess = rt.InferenceSession(onnx_model)

y_pred = np.full(shape=(len(x_train)), fill_value=np.nan)

for i in range(len(x_train)):
    inputs = {}
    for j in range(len(x_train.columns)):
        inputs[x_train.columns[j]] = np.full(shape=(1,1), fill_value=x_train.iloc[i,j])

    sess_pred = sess.run(None, inputs)
    y_pred[i] = sess_pred[0][0][0]

print(y_pred)

[49.61162567 49.61162567 49.39315414 52.83380508 49.39315414]


In [25]:
out.name

'variable1'